In [ ]:
import swisseph as swe
import json
from datetime import datetime

# Set the path to the Swiss Ephemeris data files
swe.set_ephe_path('ephe')

# Set the Ayanamsha to N.C. Lahiri
swe.set_sid_mode(swe.SIDM_LAHIRI)

# Get user input
name = input("Enter name: ")
dob = input("Enter date of birth (YYYY-MM-DD): ")
tob = input("Enter time of birth (HH:MM:SS): ")
place_of_birth = input("Enter place of birth: ")
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))

# # Dummy input values
# birth_year = "1998"
# birth_month = "9"
# birth_day = "12"
# birth_hour = "23"
# birth_minute = "49"
# district = "kannur"
# state = "kerala"
# country = "India"
# user_name = "Harshin"

# Parse the date and time of birth
year, month, day = map(int, dob.split('-'))
hour, minute, second = map(int, tob.split(':'))

# Calculate Julian Day
jd = swe.julday(year, month, day, hour + minute/60 + second/3600)

# Calculate the Ascendant using Equal house system
asc = swe.houses(jd, latitude, longitude, b'E')[0][0]

# Calculate the Rasi (Zodiac sign)
rasi = int((asc % 360) / 30)

# Calculate planetary positions and houses
planets = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto']
planet_positions = {}
planet_houses = {}
for i, planet in enumerate(planets):
    xx, ret = swe.calc_ut(jd, i, swe.FLG_SPEED)  # Include velocities in the output
    planet_positions[planet] = xx[0]  # Longitude of the planet
    print(asc)
    print(xx)
    print(type(xx[0]))
    print(latitude)
    print(longitude)
    planet_houses[planet] = swe.house_pos(asc, latitude, 'E', xx[0], xx[1])  # House of the planet

# Calculate sunrise and sunset
next_sunrise = swe.rise_trans(jd, swe.SUN, geopos=(longitude, latitude, 0), rsmi=swe.CALC_RISE)[1][0]
next_sunset = swe.rise_trans(jd, swe.SUN, geopos=(longitude, latitude, 0), rsmi=swe.CALC_SET)[1][0]

# Convert sunrise and sunset times to date-time format
next_sunrise = datetime.fromtimestamp(swe.revjul(next_sunrise)).strftime('%Y-%m-%d %H:%M:%S')
next_sunset = datetime.fromtimestamp(swe.revjul(next_sunset)).strftime('%Y-%m-%d %H:%M:%S')

# Calculate Tithi (Lunar day)
moon_long = swe.calc_ut(jd, swe.MOON)[0][0]
sun_long = swe.calc_ut(jd, swe.SUN)[0][0]
tithi = int((moon_long - sun_long) % 360 / 12) + 1

# Calculate Vara (Day of the week)
vara = int((jd + 1) % 7)
vara_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Create a dictionary to store the results
data = {
    "name": name,
    "date_of_birth": dob,
    "time_of_birth": tob,
    "place_of_birth": place_of_birth,
    "latitude": latitude,
    "longitude": longitude,
    "ascendant": asc,
    "rasi": rasi,
    "planet_positions": planet_positions,
    "planet_houses": planet_houses,
    "next_sunrise": next_sunrise,
    "next_sunset": next_sunset,
    "tithi": tithi,
    "vara": vara_names[vara]
}

# Convert the dictionary to a JSON string
json_data = json.dumps(data, indent=4)

# Print the JSON string
print(json_data)

TypeError: must be real number, not str